# Plotting Basics - IRIS-HEP Analysis Training 

Authored by: [Andrzej Novak](https://github.com/andrzejnovak), [Matthew Feickert](https://github.com/matthewfeickert)

### Histograms mean different things in different contexts
- Counts, bin edges &mdash; useful for a bar plot &mdash; `np.histogram` / `plt.bar`
- Counts, bin edges, pre computed errors &mdash; `TGraphErrors`/`plt.errorbar`
- Weighted values, weights squared, bin_edges &mdash; proper error calculation `TH1`/`Coffea.hist`/`hist`

## UHI - [Unified Histogram Interface](https://uhi.readthedocs.io/en/latest/plotting.html#using-the-protocol)
- (Plottable) Histogram protocol designed to make libraries interoperable, easy to navigate
  - Conformed to by `hist`, `mplhep`, `uproot`, `histoprint`, and now also ROOT (full UHI support in ROOT coming soon)!
- Each UHI histogram has the following methods
  - `h.values()`: The value (as given by the kind)
  - `h.variances()`: The variance in the value (None if an unweighed histogram was filled with weights)
  - `h.counts()`: How many fills the bin received or the effective number of fills if the histogram is weighted
  - `h.axes`: A Sequence of axes
  - and a few other properties

## [hist](https://github.com/scikit-hep/hist)
* Python go to one-stop for histogramming
* Extends [boost-histogram](https://github.com/scikit-hep/boost-histogram) (Python binding for C++ `Boost::Histogram` library &mdash; *FAST*)
  - Makes it user friendly
* Shortcuts for convenience and interactive plotting/fitting

## [mplhep](https://github.com/scikit-hep/mplhep)
- Built on top of `matplotlib`
- Extends functionality to easily plot histograms from various inputs
- Holds style sheets for easy experiment specific style application

We'll be using `mplhep` and `hist` the most in this tutorial, so let's make sure that the latest versions are installed in your environment. We'll also need [`scikit-hep-testdata`](https://github.com/scikit-hep/scikit-hep-testdata) and so will need to ensure it is installed in the environment.

### On Coffea-casa

```console
$ conda upgrade --yes mplhep hist
$ conda install --yes scikit-hep-testdata
```

### On your local machine in a Python virtual environment

```console
$ python -m pip install --upgrade mplhep hist scikit-hep-testdata
```

### On your local maching in a Pixi environment

```console
$ pixi upgrade hist
$ pixi add scikit-hep-testdata
```

### On your local maching in a Conda environment

```console
$ conda upgrade --yes mplhep hist
$ conda install --yes scikit-hep-testdata
```

## Tip

You can run the commands in your notebooks by prefixing with the `!` iPython magics (which escapes out to the shell). Example:

```
! conda upgrade --yes mplhep hist
```

# Outline

 - Short Matplotlib info
 - Histogramming in Matplotlib
 - `mplhep` basic example
 - `hist` basic examples and indexing with UHI
 - Analysis style example

We'll also use ROOT later on. In the event that you don't have ROOT on your machine already, you can install it from conda-forge with the following:

## In a Pixi environment

```console
$ pixi add root_base
```

## In a Conda environment

```console
$ conda install --channel conda-forge --yes root_base
```

# Two APIs of matplotlib

**References**

* [Anatomy of a [Matplotlib] figure](https://matplotlib.org/stable/gallery/showcase/anatomy.html)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
fig_dir = Path.cwd() / "figures"
fig_dir.mkdir(exist_ok=True)

Matpltolib has two [high level APIs](https://matplotlib.org/stable/api/index.html):
* The `pyplot` interface (function-based, implicit)
* The `Axes` interface (object-based, explicit)

Matplotlib as a project strongly recommends using the `Axes` API, but there are times when using the `pyplot` API is useful too.

### Stateful (`pyplot` API)

If we execute a comamnd using the `pyplot` API we see that a global state object is created

In [ ]:
plt.plot(np.arange(0, 10, 1), np.linspace(0, 1, 10))

We can continue to operate on this global state `Axes` through the `plt` API even though we aren't giving a specific `matplotlib.Axes` object to operate on

In [ ]:
plt.plot(np.arange(0, 10, 1), np.linspace(0, 1, 10))
plt.title("pyplot Example")
plt.savefig(fig_dir / "pyplot_example.png")

### Object-oriented (`Axes` API)

With the `Axes` API, we need to explicitly create a [`Figure`](https://matplotlib.org/stable/api/_as_gen/matplotlib.figure.Figure.html#matplotlib.figure.Figure) and [`Axes`](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.html#matplotlib.axes.Axes) object and then call functions that operate explicitily on the `Axes` object.

In [ ]:
fig, ax = plt.subplots()

In [ ]:
ax.plot(np.linspace(0, 1, 10), np.linspace(0, 10, 10))
ax.set_title("Axes Example")

In [ ]:
fig.savefig(fig_dir / "axes_example.png")

fig

You might notice that we still used the `pyplot` API even though we're using the `Axes` API. That's a shortcut, but you can also use the full object oriented Matplotlib API like the following

In [ ]:
from matplotlib.figure import Figure
import numpy as np

fig = Figure()
ax = fig.subplots()

x = np.linspace(0, 10, 1000)
y = np.sin(x)

ax.plot(x, y)

fig.savefig(fig_dir / "full_axes_api_example.png")

fig

## Switching back and forth

While it is generally recommended to stick with one API, it is possible to mix the two APIs in the same code

In [ ]:
fig, ax = plt.subplots()  # Axes and pyplot API
ax.stairs([1, 2, 3, 4, 3, 2, 1])  # Axes API
plt.title("Example")  # pyplot API

In [ ]:
plt.stairs([1, 2, 3, 4, 3, 2, 1])  # pyplot API
ax = plt.gca()  # Axes and pyplot API
ax.set_title("Example")  # Axes API

# Histogramming in matplotlib

Matplotlib has the concept of histograms built into it, and through the [`stairs` API](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.stairs.html) (contributed from HEP) we can arrive at something that looks familiar to us

In [ ]:
fig, ax = plt.subplots()
ax.stairs([1, 2, 3, 4, 2, 1, 0])

Visualizations of histograms can be filled with colors

In [ ]:
fig, ax = plt.subplots()
ax.stairs([1, 2, 3, 4, 2, 1, 0], baseline=0, fill=True)

Histograms can be drawn on top of each other

In [ ]:
a, b = [1, 2, 3, 4, 2, 1, 0], [1, 2, 3, 2, 2, 3, 1]

fig, ax = plt.subplots()
ax.stairs(a, label="A")
ax.stairs(b, label="B", ls="--")
ax.legend()

and `stairs` understands how to operate on [Array API](https://proceedings.scipy.org/articles/gerudo-f2bc6f59-001) compatbile objects (like NumPy arrays)

In [ ]:
fig, ax = plt.subplots()
ax.stairs(np.sum([a, b], axis=0), baseline=b, fill=True, label="A")
ax.stairs(b, fill=True, label="B")
ax.legend()

## Other histogramming methods from Matplotlib

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs = axs.flatten()

# in-situ np.histogram()
axs[0].hist(np.random.normal(5, 1, 10000))
axs[0].set_title("NumPy histogram")

# bar plots
axs[1].bar([1, 2, 3], [2, 3, 4])  # (x-position), bin-value);
axs[1].set_title("Bar plots")

# step - skyline
axs[2].step(np.arange(0, 5, 1), [2, 3, 4, 2, 1], where="post")
axs[2].set_title("step API")

# filled
axs[3].fill_between(np.arange(0, 5, 1), [2, 3, 4, 2, 1], step="post")
axs[3].set_title("fill_between API")

# Better histogramming: `mplhep`

`mplhep` gives us a high level API that allows for quickly getting the information that we traditionally think about when it comes to histograms in particle physics.

In [ ]:
import mplhep

In [ ]:
yields, bins = np.histogram(np.random.normal(5, 1, 5000), bins=10)

In [ ]:
mplhep.histplot(yields)

In [ ]:
? mplhep.histplot

### Primary goal is to stay unobtrusive, if it works in `matplotlib`, it should work in `mplhep`

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

mplhep.histplot(yields, ax=axs[0])
mplhep.histplot(yields, bins, yerr=True, ax=axs[1])
axs[1].set_title("Uncertainties shown");

### `kwargs` are passed though to `matplotlib`

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

mplhep.histplot(
    yields, ax=axs[0], histtype="fill", hatch="///", edgecolor="red", facecolor="none"
)
axs[0].set_title("Filled histogram")
mplhep.histplot(
    yields, ax=axs[1], histtype="errorbar", yerr=True, color="black", capsize=4
)
axs[1].set_title("Uncertainity outline");

### Stacking and normalizing is available

In [ ]:
bin_contents = yields.copy()
fig, axs = plt.subplots(1, 3, figsize=(18, 4))

data = np.random.poisson(bin_contents * 3)
mplhep.histplot(
    [bin_contents, bin_contents * 2],
    bins=bins,
    ax=axs[0],
    yerr=True,
    label=["MC1", "MC2"],
)
mplhep.histplot(data, bins=bins, ax=axs[1], yerr=True, label="Data")

mplhep.histplot(
    [bin_contents, bin_contents * 2],
    bins=bins,
    ax=axs[2],
    stack=True,
    label=["MC1", "MC2"],
    density=True,
)
mplhep.histplot(
    data,
    bins=bins,
    ax=axs[2],
    yerr=True,
    histtype="errorbar",
    label="Data",
    density=True,
    color="k",
)
for ax in axs:
    ax.legend()
axs[0].set_title("Some MCs")
axs[1].set_title("Draw Poisson Data")
axs[2].set_title("Data/MC Shape comparison");

### Convenient sorting options

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
mplhep.histplot(
    [bin_contents * 2, bin_contents * 3, bin_contents],
    bins=bins,
    ax=axs[0],
    stack=True,
    histtype="fill",
    label=["A", "B", "C"],
    sort="yield",
)
mplhep.histplot(
    [bin_contents * 2, bin_contents * 3, bin_contents],
    bins=bins,
    ax=axs[1],
    stack=True,
    histtype="fill",
    label=["A", "B", "C"],
    sort="label_r",
)
for ax in axs:
    ax.legend()
axs[0].set_title("Sort by yield")
axs[1].set_title("Sort by label (add _r to reverse)");

## Uproot TH1

`mplhep` is able to operate on histogram objects in ROOT files that have been deserialized with `uproot`

In [ ]:
import uproot
from skhep_testdata import data_path

file_name = data_path("uproot-hepdata-example.root")
histogram = uproot.open(file_name)
print(histogram.keys())
print(histogram["hpx"])
mplhep.histplot(histogram["hpx"]);

## PyTROOT TH1

`mplhep` also is able to operate direclty on ROOT histogram objects in PyROOT thanks to `uhi`

In [ ]:
import ROOT

histogram = ROOT.TH1F("h1", "h1", 50, -2.5, 2.5)
histogram.FillRandom("gaus", 10000)

mplhep.histplot(histogram);

# Better histogramming with `hist`

In [ ]:
import hist

First let's make just a histogram axis

In [ ]:
# histogram creation
one_axis_hist = hist.Hist(
    hist.axis.Regular(10, 0, 10, name="x", label="x-axis"), hist.storage.Int64()
)

one_axis_hist

In [ ]:
two_axes_hist = hist.Hist(
    hist.axis.Regular(10, 0, 10, name="x", label="x-axis"),
    hist.axis.Variable([0, 1, 2, 5, 10], name="y", label="y-axis"),
    hist.storage.Int64(),
)

two_axes_hist

and now let's fill it

In [ ]:
# basic filling
one_axis_hist.fill([1, 4, 6])

one_axis_hist

In [ ]:
# basic filling
two_axes_hist.fill([1, 4, 6], [3, 5, 2])

two_axes_hist

In [ ]:
# Filling by names is possible for better bookkeeping:
two_axes_hist.fill(x=[1, 5, 5, 7], y=[3, 5, 2, 7])

two_axes_hist

In [ ]:
# information access
two_axes_hist.values()

In [ ]:
two_axes_hist.axes[0]

In [ ]:
two_axes_hist.axes[0].edges

In [ ]:
# Print it (to CLI) using histoprint 'under the hood'
one_axis_hist.show(columns=50)

## Quick hist creation

Instead of having to define each axis as its own `hist.axis` object, you can also create the same histogram by chaining axes together. This is useful for quickly creating or redefining histograms.

In [ ]:
# histogram creation
h = (
    hist.new.Regular(10, 0, 10, name="x", label="x-axis")
    .Variable(range(10), name="y", label="y-axis")
    .Int64()
    .fill(*np.random.multivariate_normal([4, 6], [[2, 0], [0, 1]], 10000).T)
)

h

In [ ]:
# even quicker
h = hist.new.Reg(10, 0, 10).Var(range(10)).Int64()
# .fill(*np.random.multivariate_normal([4, 6], [[2, 0], [0, 1]], 10000).T)
h

## Axis types 

`hist` allows for [multiple kinds of axes](https://hist.readthedocs.io/en/latest/user-guide/axes.html#axis-types) from [`boost-histogram`](https://github.com/scikit-hep/boost-histogram):

* [Regular](https://hist.readthedocs.io/en/latest/user-guide/axes.html#regular-axis)
* Boolean
* [Variable](https://hist.readthedocs.io/en/latest/user-guide/axes.html#variable-axis) (variable width bins)
* Integer
* [IntCategory](https://hist.readthedocs.io/en/latest/user-guide/axes.html#category-axis) (bins correspond to categories that are indexed by integer values, e.g. `[2, 5, 7, 3, 9]`)
* [StrCategory](https://hist.readthedocs.io/en/latest/user-guide/axes.html#category-axis) (bins correspond to categories that are indexed by string values, e.g. `["Electron", "Muon"]`)

In [ ]:
axis0 = hist.axis.Regular(10, -5, 5, overflow=False, underflow=False, name="A")
axis1 = hist.axis.Boolean(name="B")
axis2 = hist.axis.Variable(range(10), name="C")
axis3 = hist.axis.Integer(-5, 5, overflow=False, underflow=False, name="D")
axis4 = hist.axis.IntCategory(range(10), name="E")
axis5 = hist.axis.StrCategory(["Electron", "Muon"], name="F", label="Particles")

In [ ]:
# Growth!
h = hist.new.Reg(10, 0, 10).StrCat([], growth=True).Weight()
h.fill(np.random.normal(5, 2, 1000), "A")
h.fill(np.random.normal(7, 2, 1000), "B")

## Storage types

A number of possible [storage types](https://hist.readthedocs.io/en/latest/user-guide/storages.html) exist: `Double`, `Unlimited`, `Int64`, `AutomicInt64`, `Weight`, `Mean`, and `WeightedMean`.

In practice you will most commonly use `Weight()` (which keeps a sum of weights)

By default, the `weight`s will be `1`

In [ ]:
hist.new.Reg(10, 0, 10).Weight().fill([1, 2, 3, 5]).plot();

and you can pass `weights` for each bin

In [ ]:
hist.new.Reg(10, 0, 10).Weight().fill([1, 2, 3, 5], weight=[1, 1, 1, 0.5]).plot();

## Hist manipulation and UHI

For mor information on `hist` check out the user guide: https://hist.readthedocs.io/

and for `uhi` the docs live at https://uhi.readthedocs.io/

In [ ]:
# example histogram
example_hist = (
    hist.new.Reg(10, 0, 10, name="x")
    .Var(range(10), name="y")
    .Var(range(10), name="z")
    .Weight()
    .fill(*np.random.multivariate_normal([4, 6, 4], np.eye(3), 100000).T)
)

example_hist

In [ ]:
# Project on an axis
example_hist.project("x")

In [ ]:
example_hist.project("y")

In [ ]:
example_hist.project("z")

In [ ]:
# Slicing (applying cuts)
# also need to project into a one or two dimensional space (summing the counts in 'z') before visualizing

# example_hist[5:, :, sum]
example_hist[5:, :, sum].plot();

By default if we index a histogram array we are indexing by _bin_. We can also use the [`j` suffix syntax to index by _value_](https://uhi.readthedocs.io/en/latest/indexing%2B.html)

In [ ]:
# Indexing by bin 5 onwards for x, and by value 6 onwards for y
example_hist[5:, 6j:, sum].plot();

Dictionary access also allows for selecting out views of the multidimensional object

In [ ]:
example_hist[5:, :, sum][{"y": 5}].plot();

and then to perform potentially complex operations

In [ ]:
example_hist[5:, :, sum][{"y": 6, "x": sum}]
# example_hist[5:, :, sum][{"y": 6, "x": 7j}]

In [ ]:
# Makes slicing inside dictionaries simpler
slicer = hist.tag.Slicer()
slicer

In [ ]:
example_hist[{"z": sum, "y": slicer[: hist.loc(5) : hist.sum]}]

# same as:
#
# example_hist[:, :, sum][{"y": 5j}]
#
# but with explicit hist APIs for slicing

## Mind the (under or over)flow bins!

In [ ]:
example_hist[sum, sum, :].values()

In [ ]:
example_hist[sum, sum, :].values(flow=True)

In [ ]:
example_hist[sum, 0:len:sum, :].values(flow=True)

In [ ]:
example_hist[sum, sum, :].values(flow=True) - example_hist[sum, 0:len:sum, :].values(
    flow=True
)

In [ ]:
# Doesn't work in dict-access
# example_hist[{0: 0:len:sum}]

In [ ]:
# Meanwhile slicer allows this syntax in dict-access
example_hist[{0: slicer[0:len:sum]}]

In [ ]:
# If you know you won't need them, you can skip flow bins
hist.new.Reg(10, 0, 10, flow=False).Weight()

 ## Hist plots with mplhep

In [ ]:
h = hist.new.Reg(10, 0, 10).Weight().fill(np.random.normal(5, 1, 1000))

In [ ]:
# Plot it
h.plot(color="red", density=True);

In [ ]:
# equivalent to
mplhep.histplot(h, color="red", density=True)

In [ ]:
# Access and modify artists
art = h.plot(color="red", density=True)
plt.setp(
    art[0].stairs, edgecolor="blue", fill=True, facecolor="lightgreen", hatch="///"
);

## N-D Histograms are cool

We can create a multi-dimensional histogram

In [ ]:
# Create a new hist
h2d = (
    hist.new.Reg(10, 0, 10, name="x")
    .StrCat(["A", "B"], growth=True, name="dataset")
    .Weight()
)
h2d

and then fill it (and expand the number of axes)

In [ ]:
h2d.fill(np.random.normal(3, 1, 1000), "A")
h2d.fill(np.random.normal(5, 1, 3000), "B")
h2d.fill(np.random.normal(7, 1, 2000), "C")
h2d.plot2d();

and also project down into a subset of the axes into different ranges

In [ ]:
h2d[:6, ["A", "B"]].plot(stack=True, histtype="step", sort="y_r")
plt.legend()

In [ ]:
mplhep.hist2dplot(h2d, labels=True);

# Analysis-like example

In [ ]:
nd_hist = (
    hist.new.Reg(100, 0, 100, name="x", label="Observable")
    .Var([0, 0.2, 0.5, 0.9, 1], name="tag", label="Some MVA")
    .StrCat(["A"], growth=True, name="dataset")
    .IntCat([0, 1, 2, 3], name="region")
    .StrCat(["A"], growth=True, name="syst", label="Systematic")
    .Weight()
)

In [ ]:
# Small random letter helper
def rnd_letters(a="A", z="Z", N=10):
    A, Z = np.array([a, z]).view("int32")
    return list(
        np.random.randint(low=A, high=Z, size=N, dtype="int32").view(f"U{N}")[0]
    )


rnd_letters("C", "F")

In [ ]:
# And fill it
N = 400000
for sample in set(rnd_letters("A", "G", 500)):
    nd_hist.fill(
        x=np.random.normal(np.random.randint(20, 80, 1), 10, N),
        tag=np.random.uniform(0, 1, N),
        dataset=sample,
        region=np.random.randint(0, 4, N),
        syst=rnd_letters("P", "Z", N=N),
    )

nd_hist

In [ ]:
# Simple slices
nd_hist[:, 0.5j:len:sum, :, 0, "X"].plot2d()

In [ ]:
# Slice by name
s = hist.tag.Slicer()
nd_hist[{"tag": s[0.5j:len:sum], "region": 0, "syst": "X"}].plot()
plt.legend();

### Scale "sample" by "cross-section"

In [ ]:
nd_hist[{"dataset": "A"}] = nd_hist[{"dataset": "A"}].view() * 2.5

nd_hist[{"tag": s[0.5j:len:sum], "region": 0, "syst": "X"}].plot()
plt.legend();

### Group datasets (to be replaced by native hist function)

In [ ]:
def groupby(h, groupmap, axis="dataset"):
    new = hist.Hist(
        *[ax for ax in h.axes if ax.name != axis],
        hist.axis.StrCategory(groupmap.keys(), name=axis, growth=True),
        hist.storage.Weight(),
    )

    for name, cats in groupmap.items():
        grouped = sum([h[{axis: name}] for name in cats])
        new[{axis: name}] = grouped.view(flow=True)
    return new


groupby(nd_hist, {"d1": ["A", "B", "C"], "d2": ["D", "E", "F"]})[
    {"tag": s[0.5j:len:sum], "region": 0, "syst": "X"}
].plot()
plt.legend();

### Desired end goal - 1D templates of each sample, passing a cut, per region per systematic

In [ ]:
cut = {"tag": s[0.5j:len:sum]}  # Events passing 0.5 threshold

templates = {}
for sample in nd_hist.axes["dataset"]:
    for region in nd_hist.axes["region"]:
        for syst in nd_hist.axes["syst"]:
            template_name = f"region{region}_{sample}_sys{syst}"
            templates[template_name] = nd_hist[
                {**cut, "dataset": sample, "region": region, "syst": syst}
            ]

In [ ]:
templates["region0_B_sysX"]

### Save it via uproot

In [ ]:
import uproot

output_file = uproot.recreate("some_file.root")
output_file["my_hist"] = templates["region0_B_sysX"]
output_file.close()

and read it back in

In [ ]:
input_file = uproot.open("some_file.root")
mplhep.histplot(input_file["my_hist"])

# Styling with mplhep
* Primary purpose of `mplhep` is to serve and distribute styles 
   - **ALICE**
   - **ATLAS**
   - **CMS**
   - **LHCb**
* To ensure plots looks the same on any framework fonts need to be included
   - I am liable to go on a rant, so suffice to say:
      - We package an open look-alike of Helvetica called Tex Gyre Heros

In [ ]:
mplhep.style.use([mplhep.style.CMS, {"figure.figsize": (8, 8)}])
mplhep.histplot(np.histogram(np.random.normal(10, 3, 1000)), histtype="fill")
mplhep.cms.label();

# CMS Colors - automatically with `hep.style.CMS`

- Data should be always shown in black. Basic color recommendations with examples are found below.

- Categorical Data (e.g. 1D Stackplots): Use the color sequence suggested by M. Petroff in [Accessible Color Sequences for Data Visualization](https://arxiv.org/abs/2107.02270) and [available on GitHub](https://github.com/mpetroff/accessible-color-cycles) (MIT License). 
- Specifically you should use the Petroff 6-color cycle:
`["#5790fc", "#f89c20", "#e42536", "#964a8b", "#9c9ca1", "#7a21dd"]`

In [ ]:
from matplotlib.colors import ListedColormap

petroff6 = ListedColormap(
    ["#5790fc", "#f89c20", "#e42536", "#964a8b", "#9c9ca1", "#7a21dd"]
)
petroff6

In [ ]:
nd_hist[
    {"tag": s[0.5j:len:sum], "region": 0, "syst": "X", "x": s[:: hist.tag.rebin(3)]}
].plot(histtype="fill", stack=True)
plt.legend();

 - or if more colors are needed the Petroff 10-color cycle:
    ```
    ["#3f90da", "#ffa90e", "#bd1f01", "#94a4a2", "#832db6", "#a96b59", "#e76300", "#b9ac70", "#717581", "#92dadd"]
    ```

which was added to Matplotlib in `v3.10.0`.

In [ ]:
# matplotlib v3.9.x
# from matplotlib.colors import ListedColormap
# petroff10 = ListedColormap(["#3f90da", "#ffa90e", "#bd1f01", "#94a4a2", "#832db6", "#a96b59", "#e76300", "#b9ac70", "#717581", "#92dadd"])
# petroff10

# maptlotlib v3.10.0+
import matplotlib.pyplot as plt

plt.style.use("petroff10")

In [ ]:
import matplotlib as mpl
from matplotlib.colors import ListedColormap

ListedColormap(mpl.color_sequences["petroff10"])

# 2D plot

In [ ]:
nd_hist[
    {"tag": s[0.5j:len:sum], "region": 0, "syst": "X", "x": s[:: hist.tag.rebin(3)]}
].plot2d();

## 2025 Improvements

### hist

`StrCat` list indexing now allows for wildcards to be used

In [ ]:
import hist
import numpy as np
import string

h = hist.new.Reg(10, 0, 10).StrCat([], growth=True).Weight()
N = 100_000
for cat in ["ABC", "BCD", "CDE", "DEF"]:
    h.fill(np.random.normal(np.random.uniform(0, 10), 1, N), cat)
h.plot2d();

Project out the axes that have an `E` in the axes name

In [ ]:
h[:, "*E*"].plot2d();

Project out the axes that have a `B` in the axes name and any axes that have an axes name that is 3 charecters long where the middle character is `D`.

### Note

If you're not familiar with the `?` wildcard syntax, it means "match exactly one charecter of any kind" where `*` means "match zero or more charecters of any kind".

In [ ]:
h[:, ["*B*", "?D?"]].plot2d();